In [10]:
import bs4
import requests
import itertools
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq
import threading
import _thread
import time
import queue
from lxml import etree
import urllib
import csv
import numpy as np
import webbrowser
import re

In [11]:
with open('AllUrls.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
my_urls = tuple(your_list)
my_urls = list(itertools.chain.from_iterable(my_urls))

In [12]:
import time
start_time = time.time()

# To get HTML elements
e_results = {}
for i in my_urls:
    uClient = uReq(i)
    page_html = uClient.read()
    e_results[i] = lh.fromstring(page_html)

print("--- %s seconds ---" % (time.time() - start_time))

--- 5426.109987974167 seconds ---


In [82]:
start_time = time.time()
df = [] #Rows
df2 = [] #Headers
for i in e_results:
    df.append(e_results[i].xpath('//td/text()'))
for i in e_results:
    df2.append(e_results[i].xpath('//th/text()'))
print("--- %s seconds ---" % (time.time() - start_time))

--- 8.83491826057434 seconds ---


In [27]:
titles = []
for i in e_results:
    titles.append(e_results[i].xpath('//tr/td/strong/text()'))
for y in range(len(titles)):
    titles[y] = [i.strip() for i in titles[y]]
    titles[y] = [i.split(':', 1)[0] for i in titles[y]]

In [78]:
uClient2 = uReq('https://www.cms.gov/apps/mds/mds_notemp/mds30FreqStart.asp?isSubmitted=mds30Freq3&var=A1000A&date=30')
page_html2 = uClient2.read()
e_results2 = lh.fromstring(page_html2)
e_results2.xpath('//td/text()')

['\r\n   \t\t\t',
 '\t\xa0\r\n   \t\t\t',
 '\r\n\t\t\t',
 '\r\n   \t\t\t    Alabama\r\n   \t\t\t',
 '\r\n   \t\t\t\t\t\t99.89%\r\n   \t\t\t\t\t',
 '\r\n   \t\t\t\t\t\t0.11%\r\n   \t\t\t\t\t',
 '\r\n\t\t\t23534\r\n   \t\t\t',
 '\r\n   \t\t\t    Alaska\r\n   \t\t\t',
 '\r\n   \t\t\t\t\t\t70.89%\r\n   \t\t\t\t\t',
 '\r\n   \t\t\t\t\t\t29.11%\r\n   \t\t\t\t\t',
 '\r\n\t\t\t632\r\n   \t\t\t',
 '\r\n   \t\t\t    Arizona\r\n   \t\t\t',
 '\r\n   \t\t\t\t\t\t92.68%\r\n   \t\t\t\t\t',
 '\r\n   \t\t\t\t\t\t7.32%\r\n   \t\t\t\t\t',
 '\r\n\t\t\t13068\r\n   \t\t\t',
 '\r\n   \t\t\t    Arkansas\r\n   \t\t\t',
 '\r\n   \t\t\t\t\t\t99.88%\r\n   \t\t\t\t\t',
 '\r\n   \t\t\t\t\t\t0.12%\r\n   \t\t\t\t\t',
 '\r\n\t\t\t17683\r\n   \t\t\t',
 '\r\n   \t\t\t    California\r\n   \t\t\t',
 '\r\n   \t\t\t\t\t\t99.66%\r\n   \t\t\t\t\t',
 '\r\n   \t\t\t\t\t\t0.34%\r\n   \t\t\t\t\t',
 '\r\n\t\t\t107943\r\n   \t\t\t',
 '\r\n   \t\t\t    Colorado\r\n   \t\t\t',
 '\r\n   \t\t\t\t\t\t99.36%\r\n   \t\t\t\t\t',
 '\r\n   \

In [5]:
for y in range(len(df)): 
    df[y] = [i.strip() for i in df[y]][3:-4]
for y in range(len(df2)):
    df2[y] = [i.strip() for i in df2[y]]

In [6]:
dfs = []
for x,y in zip(df, df2):
    cols = [x[i::len(y)] for i in range(len(y))]
    dfs.append(cols)
    # dfs.append(list(zip([col for col in cols])))

In [7]:
alldfs = []
for i in range(len(dfs)):
    alldfs.append(pd.DataFrame(dfs[i]).T)

In [8]:
df2headers = pd.DataFrame(df2)
for i in range(len(alldfs)):
    alldfs[i] = alldfs[i].rename(columns=df2headers.iloc[i])

In [9]:
checkdf = []
for i in range(len(alldfs)):
    if 'State' in alldfs[i].columns:
        checkdf.append("Yes")
    else:
        checkdf.append("No")
checkdf = [i for i, e in enumerate(checkdf) if e == 'No']

In [10]:
alldfs2 = [i for j, i in enumerate(alldfs) if j not in checkdf]

In [11]:
titles = []
for i in e_results:
    titles.append(e_results[i].xpath('//tr/td/strong/text()'))
for y in range(len(titles)):
    titles[y] = [i.strip() for i in titles[y]]
    titles[y] = [i.split(':', 1)[0] for i in titles[y]]

In [12]:
titles = [i for j, i in enumerate(titles) if j not in checkdf]

In [13]:
for i in range(len(alldfs2)):
    for y in range(len(titles)):
        alldfs2[i]['Measure'] = str(titles[i])

In [14]:
dates = []
for i in e_results:
    dates.append(e_results[i].xpath('//h4/text()'))
for y in range(len(dates)):
    dates[y] = [i.strip() for i in dates[y]]
    dates[y] = [i.split('\t\t\t', 1)[1] for i in dates[y]]
    dates[y] = [i.split(')', 1)[0] for i in dates[y]]

In [15]:
dates = [i for j, i in enumerate(dates) if j not in checkdf]

In [16]:
for i in range(len(alldfs2)):
    for y in range(len(dates)):
        alldfs2[i]['Date'] = str(dates[i])

In [18]:
for i in range(len(alldfs2)):
    alldfs2[i] = pd.melt(alldfs2[i], id_vars=['State','Date','Measure'])
maindf = pd.concat(alldfs2)

In [19]:
maindf = maindf.dropna()

In [20]:
maindf['value'] = [i.strip('%') for i in maindf['value']]

In [32]:
maindf['value'] = [i.strip('*') for i in maindf['value']]

In [33]:
maindf['value'] = pd.to_numeric(maindf['value'])

In [42]:
maindf['Date'] = maindf['Date'].replace({"['Fourth Quarter 2011']":"10/1/2011", 
                        "['First Quarter 2012']":"1/1/2012",
                        "['Second Quarter 2012']":"4/1/2012", 
                        "['Third Quarter 2012']":"7/1/2012",
                        "['Fourth Quarter 2012']":"10/1/2012", 
                        "['First Quarter 2013']":"1/1/2013",
                        "['Second Quarter 2013']":"4/1/2013", 
                        "['Third Quarter 2013']":"7/1/2013",
                        "['Fourth Quarter 2013']":"10/1/2013", 
                        "['First Quarter 2014']":"1/1/2014",   
                        "['Second Quarter 2014']":"4/1/2014", 
                        "['Third Quarter 2014']":"7/1/2014",  
                        "['Fourth Quarter 2014']":"10/1/2014", 
                        "['First Quarter 2015']":"1/1/2015",    
                        "['Second Quarter 2015']":"4/1/2015", 
                        "['Third Quarter 2015']":"7/1/2015",      
                        "['Fourth Quarter 2015']":"10/1/2015", 
                        "['First Quarter 2016']":"1/1/2016",     
                        "['Second Quarter 2016']":"4/1/2016", 
                        "['Third Quarter 2016']":"7/1/2016",      
                        "['Fourth Quarter 2016']":"10/1/2016", 
                        "['First Quarter 2017']":"1/1/2017",      
                        "['Second Quarter 2017']":"4/1/2017", 
                        "['Third Quarter 2017']":"7/1/2017",      
                        "['Fourth Quarter 2017']":"10/1/2017", 
                        "['First Quarter 2018']":"1/1/2018",      
                        "['Second Quarter 2018']":"4/1/2018", 
                        "['Third Quarter 2018']":"7/1/2018",
                        "['Fourth Quarter 2018']":"10/1/2018", 
                        "['First Quarter 2019']":"1/1/2019"})

In [45]:
maindf['Measure'] = [i.strip("['") for i in maindf['Measure']]
maindf['Measure'] = [i.strip("']") for i in maindf['Measure']]

In [ ]:
maindf.to_excel("DashboardOutput.xlsx")

In [62]:
maindf.to_csv('DashboardTXTOutput.txt', header=True, index=False, sep='\t')